<a href="https://colab.research.google.com/github/smktech9/temp/blob/main/Text_To_Video_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install
### Install Main Repo

In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/ExponentialML/Text-To-Video-Finetuning.git
%cd Text-To-Video-Finetuning/
!pip install -r requirements.txt
!pip install triton
!pip install compel
#!pip install xformers

### Install Models

In [ ]:
!git lfs install
!git clone https://huggingface.co/damo-vilab/text-to-video-ms-1.7b

### Install BLIP2 Preprocessor

In [ ]:
!git clone https://github.com/ExponentialML/Video-BLIP2-Preprocessor.git
%cd Video-BLIP2-Preprocessor/
!pip install -r requirements.txt

fatal: destination path 'Video-BLIP2-Preprocessor' already exists and is not an empty directory.
/content/drive/MyDrive/Text-To-Video-Finetuning/Video-BLIP2-Preprocessor
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Preprocess Videos
See arguments you can pass [here](https://github.com/ExponentialML/Video-BLIP2-Preprocessor#default-arguments)

In [ ]:
%cd /content/drive/MyDrive/Text-To-Video-Finetuning/Video-BLIP2-Preprocessor
!python preprocess.py --help

In [ ]:
%cd /content/drive/MyDrive/Text-To-Video-Finetuning/Video-BLIP2-Preprocessor
!python preprocess.py \
--video_directory /content/drive/MyDrive/secrets-flowers-blip2 \
--config_name "secrets" \
--config_save_name "secrets" \
--prompt_amount 8


## Train Video

In [ ]:
%%writefile /content/drive/MyDrive/Text-To-Video-Finetuning/configs/my_config_hq.yaml

pretrained_model_path: "/content/drive/MyDrive/Text-To-Video-Finetuning/text-to-video-ms-1.7b" #https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/tree/main
output_dir: "./outputs"
train_text_encoder: False
#resume_from_checkpoint: None

train_data:
  json_path: "/content/drive/MyDrive/Text-To-Video-Finetuning/Video-BLIP2-Preprocessor/train_data/secrets.json"
  preprocessed: True
  n_sample_frames: 48
  shuffle_frames: False
  width: 384
  height: 256
  sample_start_idx: 0
  sample_frame_rate: 24
  vid_data_key: "video_path"

  single_video_path: ""
  single_video_prompt: ""

validation_data:
  prompt: ""
  sample_preview: True
  num_frames: 16
  width: 384
  height: 256
  num_inference_steps: 50
  guidance_scale: 9

learning_rate: 1e-5
adam_weight_decay: 1e-2
train_batch_size: 1
max_train_steps: 25000
checkpointing_steps: 2500
validation_steps: 250
trainable_modules:
  - "attentions"
seed: 64
mixed_precision: "fp16"
use_8bit_adam: False # This seems to be incompatible at the moment.

# Xformers must be installed
enable_xformers_memory_efficient_attention: False


Overwriting /content/drive/MyDrive/Text-To-Video-Finetuning/configs/my_config_hq.yaml


In [ ]:
%cd /content/drive/MyDrive/Text-To-Video-Finetuning
!python train.py --config ./configs/my_config_hq.yaml

In [ ]:
from google. colab import runtime
runtime.unassign()

## Inference

In [ ]:
%cd /content/drive/MyDrive/Text-To-Video-Finetuning

/content/drive/MyDrive/Text-To-Video-Finetuning


In [ ]:
import random
import numpy as np
import torch
from diffusers import DPMSolverMultistepScheduler, DDPMScheduler, TextToVideoSDPipeline
from diffusers.utils import export_to_video
from compel import Compel
import imageio

def to_video(fn: str, frames: list[np.ndarray], fps: int) -> str:
    # out_file = tempfile.NamedTemporaryFile(suffix='.mp4', delete=False)
    writer = imageio.get_writer(fn, format='FFMPEG', fps=fps)
    for frame in frames:
        writer.append_data(frame)
    writer.close()
    return fn

my_trained_model_path = "/content/drive/MyDrive/Text-To-Video-Finetuning/outputs/train_2023-03-29T00-33-16/checkpoint-15000"
pipe = TextToVideoSDPipeline.from_pretrained(my_trained_model_path,torch_dtype=torch.float16, variant="fp16")
compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()


seed = random.randint(0, 1000000)
generator = torch.Generator().manual_seed(seed)

prompt = "a close up of a red flower in a vase against a (blue background), beautiful flowers, trending on artstation"
neg_prompt = "watermark+++, text, shutterstock text, shutterstock++, blurry, ugly, username, url, low resolution, low quality"
prompt_embeds = compel.build_conditioning_tensor(prompt)
neg_prompt_embeds = compel.build_conditioning_tensor(neg_prompt)
video_frames = pipe(prompt_embeds=prompt_embeds, negative_prompt_embeds=neg_prompt_embeds, num_frames=100, width=384, height=256, num_inference_steps=50, guidance_scale=9, generator=generator).frames

prompt_string = prompt.replace(" ", "-")
out_file = "/content/"+prompt_string+"-seed"+str(seed)+".mp4"
video_path = to_video(out_file, video_frames, 24)
#video_path = export_to_video(video_frames, out_file)
print(video_path+ " is ready")

  0%|          | 0/50 [00:00<?, ?it/s]

/content/a-close-up-of-a-red-flower-in-a-vase-against-a-(blue-background),-beautiful-flowers,-trending-on-artstation-seed544798.mp4 is ready


### Batch inference
This will run on all of the prompts generate by BLIP2 and create one video for each.

In [ ]:
import json
import random
import numpy as np
import torch
from diffusers import DPMSolverMultistepScheduler, DDPMScheduler, TextToVideoSDPipeline
from diffusers.utils import export_to_video
import imageio

def to_video(fn: str, frames: list[np.ndarray], fps: int) -> str:
    # out_file = tempfile.NamedTemporaryFile(suffix='.mp4', delete=False)
    writer = imageio.get_writer(fn, format='FFMPEG', fps=fps)
    for frame in frames:
        writer.append_data(frame)
    writer.close()
    return fn

my_trained_model_path = "/content/drive/MyDrive/Text-To-Video-Finetuning/outputs/train_2023-03-29T14-37-50/checkpoint-2500"
pipe = TextToVideoSDPipeline.from_pretrained(my_trained_model_path,torch_dtype=torch.float16, variant="fp16")
compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

jsonf = open("/content/drive/MyDrive/Text-To-Video-Finetuning/Video-BLIP2-Preprocessor/train_data/secrets.json")
data = json.load(jsonf)
for d in reversed(data['data']):
  for d2 in d['data']:
    #prompt = d2['prompt'].replace('flower', 'mushroom flower').replace('flowers', 'mushroom flowers')
    #prompt = d2['prompt'].replace('flowers', 'skull made of flowers').replace('flower', 'skull made of flowers')
    #prompt = d2['prompt'].replace('orange', 'pink').replace('yellow', 'pink').replace('purple','yellow')
    prompt = d2['prompt']
    neg_prompt = "watermark, text, shutterstock text, shutterstock, blurry, ugly, low res, low quality, username, url, flower"
    prompt_embeds = compel.build_conditioning_tensor(prompt)
    neg_prompt_embeds = compel.build_conditioning_tensor(neg_prompt)
    seed = random.randint(0, 1000000)
    generator = torch.Generator().manual_seed(seed)
    video_frames = pipe(prompt, negative_prompt=neg_prompt, num_frames=120, width=384, height=256, num_inference_steps=40, guidance_scale=9, generator=generator).frames

    prompt_string = prompt.replace(" ", "-").replace(":/","").replace("/","")
    out_file = "/content/"+prompt_string+"-seed"+str(seed)+".mp4"
    video_path = to_video(out_file, video_frames, 18)
    #video_path = export_to_video(video_frames, out_file)
    print(video_path+ " is ready")


  0%|          | 0/40 [00:00<?, ?it/s]

/content/an-image-of-a-pink-lily-with-red-stamens-and-green-leaves-seed945870.mp4 is ready


  0%|          | 0/40 [00:00<?, ?it/s]

/content/a-close-up-of-a-pink-flower-with-red-stamens-and-petals-seed483977.mp4 is ready


  0%|          | 0/40 [00:00<?, ?it/s]

...

/content/a-close-up-of-a-flower-with-pink-petals-and-leaves-on-it-seed537275.mp4 is ready


  0%|          | 0/40 [00:00<?, ?it/s]

/content/a-close-up-of-a-plant-with-pink-flowers-and-green-leaves-on-it-seed135032.mp4 is ready


  0%|          | 0/40 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 24>:33                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/diffusers/pipelines/text_to_video_synthesis/pipeline_text │
│ _to_video_synth.py:634 in __call__                                                               │
│                                                                                                  │
│   631 │   │   │   │   latent_model_input = self.scheduler.scale_model_input(latent_model_input   │
│   632 │   │   │   │                                                                              │
│   633 │   │   │   │   # predict the noise residual                                               │
│ ❱ 634 │   │   │   │   noise_pred = self.unet(                                                    │
│   635 │   │   │   │   │   latent_model_input,                                                    │
│   636 │   │   │   │   │   t,                                                                     │
│   637 │   │   │   │   │   encoder_hidden_states=prompt_embeds,                                   │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1194 in _call_impl             │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/accelerate/hooks.py:165 in new_forward                    │
│                                                                                                  │
│   162 │   │   │   with torch.no_grad():                                                          │
│   163 │   │   │   │   output = old_forward(*args, **kwargs)                                      │
│   164 │   │   else:                                                                              │
│ ❱ 165 │   │   │   output = old_forward(*args, **kwargs)    

In [ ]:
!mkdir /content/skull-videos

In [ ]:
!mv /content/*.mp4 /content/skull-videos

In [ ]:
!zip -r  skull-videos.zip /content/skull-videos

  adding: content/skull-videos/ (stored 0%)
  adding: content/skull-videos/a-yellow-dandelion-skull-made-of-flowers-against-a-blue-sky-stock-video-videos-&-royalty-free-footage-seed869540.mp4 (deflated 0%)
  adding: content/skull-videos/a-yellow-dandelion-skull-made-of-flowers-against-a-blue-sky-stock-video-footage-and-b-roll-footage-seed333256.mp4 (deflated 0%)
  adding: content/skull-videos/a-close-up-of-an-orange-skull-made-of-flowers-in-a-vase-against-a-blue-sky-seed800814.mp4 (deflated 0%)
  adding: content/skull-videos/a-yellow-dandelion-skull-made-of-flowers-against-a-blue-sky-stock-video-videos-&-royalty-free-footage-seed498948.mp4 (deflated 0%)
  adding: content/skull-videos/vídeos-de-stock-e-filmes-b-roll-stock-footage-&-royalty-free-video-footage-seed374108.mp4 (deflated 0%)
  adding: content/skull-videos/an-orange-tiger-lily-with-brown-spots-on-it's-stamens-seed34974.mp4 (deflated 0%)
  adding: content/skull-videos/an-orange-hibiscus-skull-made-of-flowers-is-shown-in-this-o

In [ ]:
from google.colab import files


files.download('/content/mushroom-videos.zip')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/files.py:222 in download                     │
│                                                                                                  │
│   219                                                                                            │
│   220   if not _os.path.exists(filename):                                                        │
│   221 │   msg = 'Cannot find file: {}'.format(filename)                                          │
│ ❱ 222 │   raise FileNotFoundError(msg)  # pylint: disable=undefined-variable                     │
│   223                                                                                            │
│   224   comm_manager = _IPython.get_ipython().kernel.comm_manager                                │
│   225   comm_id = 'download_' + str(_uuid.uuid4())                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: Cannot find file: /content/mushroom-videos.zip

In [ ]:
from google.colab import runtime
runtime.unassign()